In [1]:
## Funciones de Usuario (udfs)
##   Apache Pig permite la ejecución de funciones escritas en Java, Jython (http://www.jython.org/index.html), JavaScript, Ruby, Groovy y Python. 
##   Información detallada de la ejecución de udfs en estos lenguajes puede ser obtenida en https://pig.apache.org/docs/latest/udf.html#python-udfs.
##   Véase http://www.jython.org/index.html. Para su uso, el script de Python debe estar ubicado en el directorio de trabajo. Para poder usar Jython, hay dos opciones; 
##   en la primera se debe indicar la ubicación del inteprete en la variable PIG_CLASSPATH. En algunos casos es necesario establacer la variable PYTHONPATH para que 
##   Pig pueda compilar las udfs.

In [15]:
%load_ext bigdata

The bigdata extension is already loaded. To reload it, use:
  %reload_ext bigdata


In [16]:
%pig_start

In [17]:
%timeout 300

In [4]:
%%writefile udf.tsv
A	B	1
C	D	2
E	F	3
G	H	4
I	J	5

Writing udf.tsv


In [5]:
!hadoop fs -put udf.tsv

In [6]:
%%writefile pigudf.py

#from pig_util import outputSchema

@outputSchema("as:int")
def square(num):
    if num == None:
        return None
    return ((num) * (num))

@outputSchema("word:chararray")
def concatenar(word1, word2):
    return (word1 + word2)

Writing pigudf.py


In [7]:
%%pig
--
-- registra las funciones en el script `pigudf.py`.
--
REGISTER /Users/jdvelasq/jython2.7.0/jython.jar
REGISTER 'pigudf.py' using jython as myudf;

u = LOAD 'udf.tsv' USING PigStorage()
    AS (c1:CHARARRAY,
        c2:CHARARRAY,
        c3:INT);

b = FOREACH u GENERATE myudf.concatenar(c1, c2), myudf.square(c3);
DUMP b;

 --
 -- registra las funciones en el script `pigudf.py`.
 --
 REGISTER /Users/jdvelasq/jython2.7.0/jython.jar
2019-12-07 13:45:43,627 [main] ERROR org.apache.pig.tools.grunt.Grunt - ERROR 101: file '/Users/jdvelasq/jython2.7.0/jython.jar' does not exist.
Details at logfile: /datalake/pig_1575726330038.log
 REGISTER 'pigudf.py' using jython as myudf;
 u = LOAD 'udf.tsv' USING PigStorage()
    AS (c1:CHARARRAY,
        c2:CHARARRAY,
        c3:INT);
 b = FOREACH u GENERATE myudf.concatenar(c1, c2), myudf.square(c3);
 DUMP b;
2019-12-07 13:45:47,517 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2019-12-07 13:45:47,616 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.reduce.markreset.buffer.percent is deprecated. Instead, use mapreduce.reduce.markreset.buffer.percent
2019-12-07 13:45:47,617 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.output.compress is deprecated. Instead, use mapredu

In [8]:
!rm pigudf.py pig_*  udf.tsv

In [9]:
%%writefile transacciones.txt
1	1	1	300	a jumper
2	1	2	300	a jumper
3	1	2	300	a jumper
4	2	3	100	a rubber chicken
5	1	3	300	a jumper
6	1	4	300	a jumper
7	1	4	250	a jumper

Writing transacciones.txt


In [10]:
%%writefile usuarios.txt
1	matthew@test.com	EN	US
2	matthew@test2.com	EN	GB
3	matthew@test3.com	FR	FR
4	matthew@test2.com	ES	GB

Writing usuarios.txt


In [11]:
!hadoop fs -put transacciones.txt
!hadoop fs -put usuarios.txt

In [12]:
!hadoop fs -ls

Found 2 items
-rw-r--r--   1 root supergroup        141 2019-12-07 14:39 transacciones.txt
-rw-r--r--   1 root supergroup        103 2019-12-07 14:39 usuarios.txt


In [21]:
%%pig
u = LOAD 'usuarios.txt' USING PigStorage()
    AS (userid:INT,
        email:CHARARRAY,
        lang:CHARARRAY,
        location: CHARARRAY);
    
t = LOAD 'transacciones.txt' USING PigStorage()
    AS (tranid:INT,
        prodid:INT,
        userid:INT,
        valorventa: INT,
        nombre: CHARARRAY);

TempR4 = JOIN u BY userid, t BY userid;
r4 = FOREACH TempR4 GENERATE prodid, nombre, location;
DUMP r4;

 u = LOAD 'usuarios.txt' USING PigStorage()
    AS (userid:INT,
        email:CHARARRAY,
        lang:CHARARRAY,
        location: CHARARRAY);
 t = LOAD 'transacciones.txt' USING PigStorage()
    AS (tranid:INT,
        prodid:INT,
        userid:INT,
        valorventa: INT,
        nombre: CHARARRAY);
 TempR4 = JOIN u BY userid, t BY userid;
 r4 = FOREACH TempR4 GENERATE prodid, nombre, location;
 DUMP r4;
2019-12-07 14:56:13,830 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2019-12-07 14:56:14,350 [JobControl] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2019-12-07 14:56:14,357 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2019-12-07 14:56:14,362 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2019-12-07 14:56:14,364 [

In [28]:
%%pig
TempR5 = JOIN u BY userid, t BY userid;
TempR5_2 = FOREACH TempR5 GENERATE location, valorventa;
TempR5_3 = GROUP TempR5_2 BY location;
r5 = FOREACH TempR5_3 GENERATE group, SUM(TempR5_2.valorventa);
DUMP r5;

 TempR5 = JOIN u BY userid, t BY userid;
 TempR5_2 = FOREACH TempR4 GENERATE location, valorventa;
 TempR5_3 = GROUP TempR5_2 BY location;
 r5 = FOREACH TempR5_3 GENERATE group, SUM(TempR5_2.valorventa);
 DUMP r5;
2019-12-07 15:22:24,322 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2019-12-07 15:22:24,796 [JobControl] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2019-12-07 15:22:24,807 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2019-12-07 15:22:24,817 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2019-12-07 15:22:24,819 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2019-12-07 15:22:25,252 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - numbe

In [29]:
%%pig
TempR5 = JOIN u BY userid, t BY userid;
TempR5_2 = FOREACH TempR5 GENERATE lang,location, valorventa;
TempR5_3 = GROUP TempR5_2 BY (lang,location);
r5 = FOREACH TempR5_3 GENERATE group, SUM(TempR5_2.valorventa);
DUMP r5;

 TempR5 = JOIN u BY userid, t BY userid;
 TempR5_2 = FOREACH TempR5 GENERATE lang,location, valorventa;
 TempR5_3 = GROUP TempR5_2 BY (lang,location);
 r5 = FOREACH TempR5_3 GENERATE group, SUM(TempR5_2.valorventa);
 DUMP r5;
2019-12-07 15:42:10,155 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2019-12-07 15:42:11,442 [JobControl] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2019-12-07 15:42:11,454 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2019-12-07 15:42:11,463 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2019-12-07 15:42:11,466 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2019-12-07 15:42:11,906 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmi

In [34]:
%%pig
r6 = FOREACH r5 GENERATE FLATTEN($0), $1;
DUMP r6;

 r6 = FOREACH r5 GENERATE FLATTEN($0), $1;
 DUMP r6;
2019-12-07 15:45:15,009 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2019-12-07 15:45:15,069 [JobControl] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2019-12-07 15:45:15,077 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2019-12-07 15:45:15,081 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2019-12-07 15:45:15,083 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2019-12-07 15:45:15,520 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - number of splits:2
2019-12-07 15:45:15,536 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - Submitting tokens for job: job_1575720238556_0015
2019-12-07 

In [43]:
%%writefile ejercicio2.py

#from pig_util import outputSchema

@outputSchema("as:int")
def impuestos(pais, valorven):
    if pais == None:
        return None
    if pais == 'US':
        return valorven * 0.08
    if pais == 'FR':
        return valorven * 0.12
    if pais == 'GB':
        return valorven * 0.18
    return 0


Overwriting ejercicio2.py


In [37]:
%%pig
DESCRIBE u;
DESCRIBE t;

 DESCRIBE u;
u: {
    userid: int,
    email: chararray,
    lang: chararray,
    location: chararray
}
 DESCRIBE t;
t: {
    tranid: int,
    prodid: int,
    userid: int,
    valorventa: int,
    nombre: chararray
}


In [46]:
%%pig
REGISTER /Users/jdvelasq/jython2.7.0/jython.jar
REGISTER 'ejercicio2.py' using jython as myudf2;

TempR5 = JOIN u BY userid, t BY userid;
TempR5_2 = FOREACH TempR5 GENERATE tranid,nombre,location, valorventa;
tmpR7 = FOREACH TempR5_2 GENERATE trainid,nombre,valorventa,myudf2.impuestos(location, valorventa) as valorimpuestos,valorventa+myudf2.impuestos(location, valorventa) as valortotal;
DUMP tmpR7;


 REGISTER /Users/jdvelasq/jython2.7.0/jython.jar
2019-12-07 16:18:51,685 [main] ERROR org.apache.pig.tools.grunt.Grunt - ERROR 101: file '/Users/jdvelasq/jython2.7.0/jython.jar' does not exist.
Details at logfile: /datalake/pig_1575730367516.log
 REGISTER 'ejercicio2.py' using jython as myudf2;
 TempR5 = JOIN u BY userid, t BY userid;
 TempR5_2 = FOREACH TempR5 GENERATE tranid,nombre,location, valorventa;
 tmpR7 = FOREACH TempR5_2 GENERATE trainid,nombre,valorventa,
myudf2.impuestos(location, valorventa) as valorimpuestos,
valorventa+myudf2.impuestos(location, valorventa) as valortotal;
2019-12-07 16:18:52,263 [main] ERROR org.apache.pig.tools.grunt.Grunt - ERROR 1025: <line 83, column 34> Invalid field projection. Projected field [trainid] does not exist in schema: t::tranid:int,t::nombre:chararray,u::location:chararray,t::valorventa:int.
Details at logfile: /datalake/pig_1575730367516.log
 DUMP tmpR7;
2019-12-07 16:18:52,345 [main] ERROR org.apache.pig.tools.grunt.Grunt - ERROR 1003: